$$
\newcommand{\x}{\mathbf{x}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\price}{{p}}
\newcommand{\ret}{{r}}
\newcommand{\tp}{{(t)}}
\newcommand{\aapl}{{\text{AAPL}}}
\newcommand{\ba}{{\text{BA}}}
\newcommand{\spy}{{\text{SPY}}}
$$

# Assignment: Using Machine Learning for Hedging

Welcome to the first assignment.  

We will show how Machine Learning can be used in Finance to build multi-asset portfolios that have better risk/return characteristics than a portfolio consisting of a single asset.

# Objectives
We will be using Linear Regression to establish the relationship between the returns of individual equities and "the market".

The purpose of the assignment is two-fold
- to get you up to spped with Machine Learning in general, and `sklearn` in particular
- to get you up to speed with the other programming tools (e.g., Pandas) that will help you in data preparation, etc.

# How to report your answers
I will mix explanation of the topic with tasks that you must complete. Look for 
the string "**Queston**" to find a task that you must perform.
Most of the tasks will require you to assign values to variables and execute a `print` statement.

**Motivation**

If you *do not change* the print statement then the GA (or a machine) can automatically find your answer to each part by searching for the string.


# The data

The data are the daily prices of a number of individual equities and equity indices.
The prices are arranged in a series in ascending date order (a timeseries).
- There is a separate `.csv` file for each equity or index in the directory `data/assignment_1`

## Reading the data

You should get the price data into some sort of data structure.  Pandas DataFrame is super useful
so I recommend that's what you use (not required though).

**Hints**: 
- look up the Pandas `read_csv` method
- it will be very convenient to use dates as the index of your DataFrame

## Preliminary data preparation

In the rest of the assignment we will *not* be working with prices but with *returns* (percent change in prices).
For example, for ticker $\aapl$ (Apple)

$$
\begin{array}[lll]\\
\ret_\aapl^\tp = \frac{\price _\aapl^\tp}{\price _\aapl^{(t-1)}} -1 \\
\text{where} \\
\price_\aapl^\tp \text{ denotes the price of ticker } \aapl \text{ on date } t \\
\ret_\aapl^\tp \text{ denotes the return of ticker } \aapl \text{ on date } t
\end{array}
$$

- You will want to convert the price data into return data
- We only want the returns for the year 2018; discard any other return

**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [9]:
import pandas as pd

# Load data from the CSV file
df = pd.read_csv("A1csv files/AAPL.csv", parse_dates=["Dt"])

# Sort by date
df = df.sort_values("Dt")

# Compute daily returns
df["Return"] = df["Close"].pct_change()

# Filter for 2018
df_2018 = df[df["Dt"].dt.year == 2018]

# Compute required values in one line each
num_returns = len(df_2018)  # Number of returns in 2018
first_return = df_2018["Return"].iloc[0] if not df_2018.empty else 0  # First return in 2018
last_return = df_2018["Return"].iloc[-1] if not df_2018.empty else 0  # Last return in 2018
avg_return = df_2018["Return"].mean() if not df_2018.empty else 0  # Average return in 2018
print(df_2018)

print("There are {num:d} returns. First={first:3.2f}, Last={last:3.2f}, Avg={avg:3.2f}".format(num=num_returns, first=first_return, last=last_return, avg=avg_return))

            Dt   Close    Return
3   2018-01-02  172.26  0.017905
4   2018-01-03  172.23 -0.000174
5   2018-01-04  173.03  0.004645
6   2018-01-05  175.00  0.011385
7   2018-01-08  174.35 -0.003714
..         ...     ...       ...
249 2018-12-24  146.83 -0.025874
250 2018-12-26  157.17  0.070422
251 2018-12-27  156.15 -0.006490
252 2018-12-28  156.23  0.000512
253 2018-12-31  157.74  0.009665

[251 rows x 3 columns]
There are 251 returns. First=0.02, Last=0.01, Avg=-0.00


# Split into Train and Test datasets

In general, you will split the data into two sets by choosing the members of each set at random.

To facilitate grading for this assignment, we will *use a specific test set*
- the training set are the returns for the months of January through September (inclusive), i.e., 9 months
- the test set are the returns for the months of October through December (inclusive), i.e., 3 months

Thus, you will be using the early part of the data for training, and the latter part of the data for testing.

**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [18]:
import pandas as pd

# Load data from CSV
df = pd.read_csv("A1csv files/AAPL.csv", parse_dates=["Dt"])

# Sort by date to ensure chronological order
df = df.sort_values("Dt")

# Compute daily returns
df["Return"] = df["Close"].pct_change()

# Filter data for **only the year 2018**
df_2018 = df[df["Dt"].dt.year == 2018].dropna()  # Drops NaN from pct_change()

# Split into training (Jan-Sep) and testing (Oct-Dec) sets
train_df = df_2018[(df_2018["Dt"].dt.month >= 1) & (df_2018["Dt"].dt.month <= 9)]  
test_df = df_2018[(df_2018["Dt"].dt.month >= 10) & (df_2018["Dt"].dt.month <= 12)]  

# Compute statistics for training set
train_num_returns = len(train_df)  
train_first_return = train_df["Return"].iloc[0] if not train_df.empty else 0  
train_last_return = train_df["Return"].iloc[-1] if not train_df.empty else 0  
train_avg_return = train_df["Return"].mean() if not train_df.empty else 0  

print("Training set: There are {num:d} returns. First={first:3.4f}, Last={last:3.4f}, Avg={avg:3.4f}".format(
    num=train_num_returns, first=train_first_return, last=train_last_return, avg=train_avg_return))

# Compute statistics for test set
test_num_returns = len(test_df)  
test_first_return = test_df["Return"].iloc[0] if not test_df.empty else 0  
test_last_return = test_df["Return"].iloc[-1] if not test_df.empty else 0  
test_avg_return = test_df["Return"].mean() if not test_df.empty else 0  

print("Test set: There are {num:d} returns. First={first:3.4f}, Last={last:3.4f}, Avg={avg:3.4f}".format(
    num=test_num_returns, first=test_first_return, last=test_last_return, avg=test_avg_return))


Training set: There are 188 returns. First=0.0179, Last=0.0035, Avg=0.0016
Test set: There are 63 returns. First=0.0067, Last=0.0097, Avg=-0.0054


# $\aapl$ regression

Use Linear Regression to predict the return of a ticker from the return of the $\spy$ index.
For example, for ticker $\aapl$

$$
\ret_\aapl^\tp =  \beta_{\aapl, \spy} * \ret_\spy^\tp + \epsilon_{\aapl}^\tp
$$

That is
- each example is a pair consisting of one day's return 
    - of the ticker (e.g., $\aapl$).  This is the target (e.g, $\y$ in our lectures)
    - of the index $\spy$. This is a feature vector of length 1 (e.g., $\x$ in our lectures)

You will use Linear Regression to solve for parameter $\beta_{\aapl, \spy}$ 

- In the lectures we used the symbol $\Theta$ to denote the parameter vector; here we use $\mathbf{\beta}$
- In Finance the symbol $\beta$ is often used to denote the relationship between returns. 
- You may should add an "intercept" so that the feature vector is length 2 rather than length 1
    - $\x^\tp = \begin{pmatrix}
        1 \\
        \ret_\spy^\tp
        \end{pmatrix}$




- Report the $\mathbf{\beta}$ parameter vector you obtain for $\aapl$
    - you will subsequently do this for another ticker in a different part of the assignment
        - so think ahead: you may want to parameterize your code
        - change the assignment to `ticker` when you report the next part

        
**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements


In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

def load_and_preprocess(stock_path, benchmark_path):
    """ Load stock & benchmark data, compute daily returns, and merge them. """
    stock_df = pd.read_csv(stock_path, parse_dates=["Dt"])
    benchmark_df = pd.read_csv(benchmark_path, parse_dates=["Dt"])

    # Sort by date
    stock_df = stock_df.sort_values("Dt")
    benchmark_df = benchmark_df.sort_values("Dt")

    # Compute daily returns
    stock_df["Return"] = stock_df["Close"].pct_change()
    benchmark_df["Return"] = benchmark_df["Close"].pct_change()

    # Merge data on date
    merged_df = pd.merge(stock_df[["Dt", "Return"]], benchmark_df[["Dt", "Return"]], on="Dt", suffixes=("_Stock", "_Benchmark"))

    # Filter for 2018 & drop NaN values
    df_2018 = merged_df[merged_df["Dt"].dt.year == 2018].dropna()

    return df_2018

def compute_regression(df):
    """ Perform linear regression to estimate beta parameters. """
    X = df[["Return_Benchmark"]].values  # Feature: Benchmark return
    y = df["Return_Stock"].values        # Target: Stock return

    # Add intercept term
    X = np.c_[np.ones(X.shape[0]), X]

    # Fit Linear Regression model
    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)

    # Extract parameters
    beta_0, beta_stock = model.coef_

    return beta_0, beta_stock, model, X, y

def compute_cross_val_score(model, X, y, ticker):
    """ Compute and print the 5-fold cross-validation score. """
    cv_scores = cross_val_score(model, X, y, cv=5, scoring="r2")
    cross_val_avg = np.mean(cv_scores)

    print("{t:s}: Avg cross val score = {sc:3.2f}".format(t=ticker, sc=cross_val_avg))

    return cross_val_avg  # In case we need the value later

def run_analysis(stock_path, benchmark_path, ticker):
    """ Full pipeline: Load data, compute regression, and cross-validation. """
    df_2018 = load_and_preprocess(stock_path, benchmark_path)

    # Compute regression
    beta_0, beta_stock, model, X, y = compute_regression(df_2018)

    # Print beta values
    print("{t:s}: beta_0={b0:3.2f}, beta_Benchmark={b1:3.2f}".format(t=ticker, b0=beta_0, b1=beta_stock))

    
ticker = 'APPL'
# Run for AAPL using SPY as benchmark
run_analysis("A1csv files/AAPL.csv", "A1csv files/SPY.csv", "AAPL")

# β_0 = 0 is logical, since AAPL’s return is mostly dependent on market returns
# AAPL moves 1.26x more than SPY, which is expected for a high-growth tech stock

AAPL: beta_0=0.00, beta_Benchmark=1.26


- Report the average of the cross validation scores, using 5 fold cross validation

**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements


In [46]:
cross_val_avg = np.mean(cv_scores)

print("{t:s}: Avg cross val score = {sc:3.2f}".format(t=ticker, sc=cross_val_avg) )
# 46% of its movement is explained by SPY, meaning the market is a big influence, but not the only one

AAPL: Avg cross val score = 0.46


## $\aapl$ hedged returns

- Compute the series
$$
{\ret'}_{\aapl}^\tp = \ret_\aapl^\tp - \beta_{\aapl, \spy} * \ret_\spy^\tp
$$
for all dates $t$ in the test set.  
- Sort the dates in ascending order and plot the timeseries ${\ret}'_{\aapl}$

${\ret}'_{\aapl}$ is called the "hedged return" of $\aapl$
- It is the daily return you would realize if you created a portfolio that was
    - long 1 dollar of $\aapl$
    - short $\beta_{\aapl, \spy}$ dollars of the index $\spy$
- It represents the outperformance of $\aapl$ relative to the index $\spy$
    - $\spy$ is the proxy for "the market" (it tracks the S&P 500 index)
    - The hedged return is the *value added* by going long $\aapl$ rather than just going "long the market"
    - Sometimes referred to as the "alpha" ($\alpha_\aapl$)
- So **if** you are able to correctly forecast that $\aapl$ will have positive outperformance (i.e, have $\alpha_\aapl > 0$ most days)
    - then you can earn a positive return regardless of whether the market ($\spy$) goes up or down !
    - this is much lower risk than just holding $\aapl$ long
    - people will pay you very well if you can really forecast correctly !

**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [39]:
import pandas as pd
import numpy as np

def load_and_preprocess(stock_path, benchmark_path):
    """ Load stock & benchmark data, compute daily returns, and merge them. """
    stock_df = pd.read_csv(stock_path, parse_dates=["Dt"])
    benchmark_df = pd.read_csv(benchmark_path, parse_dates=["Dt"])

    # Sort by date
    stock_df = stock_df.sort_values("Dt")
    benchmark_df = benchmark_df.sort_values("Dt")

    # Compute daily returns
    stock_df["Return"] = stock_df["Close"].pct_change()
    benchmark_df["Return"] = benchmark_df["Close"].pct_change()

    # Merge data on date
    merged_df = pd.merge(stock_df[["Dt", "Return"]], benchmark_df[["Dt", "Return"]], on="Dt", suffixes=("_Stock", "_Benchmark"))

    # Filter for 2018 & drop NaN values
    df_2018 = merged_df[merged_df["Dt"].dt.year == 2018].dropna()

    return df_2018

def compute_hedged_returns(df, beta_stock, ticker):
    """ Compute hedged returns: r'_Stock = r_Stock - beta * r_Benchmark """
    df.loc[:, "Hedged_Return"] = df["Return_Stock"] - (beta_stock * df["Return_Benchmark"])

    hedged_num_returns = len(df)
    hedged_first_return = df["Hedged_Return"].iloc[0] if not df.empty else 0
    hedged_last_return = df["Hedged_Return"].iloc[-1] if not df.empty else 0
    hedged_avg_return = df["Hedged_Return"].mean() if not df.empty else 0

    print("{t:s} hedged returns: There are {num:d} returns. First={first:3.4f}, Last={last:3.4f}, Avg={avg:3.4f}".format(
        t=ticker, num=hedged_num_returns, first=hedged_first_return, last=hedged_last_return, avg=hedged_avg_return))
    
    return df

# Load data
df_2018 = load_and_preprocess("A1csv files/AAPL.csv", "A1csv files/SPY.csv")

# Use previously computed beta (β_AAPL,SPY)
beta_SPY = 1.26  # From regression

# Compute hedged returns
df_2018 = compute_hedged_returns(df_2018, beta_SPY, "AAPL")


AAPL hedged returns: There are 251 returns. First=0.0089, Last=-0.0014, Avg=0.0001


# $\ba$ regression

Repeat the regression you carried out for $\aapl$ but this time instead for the ticker $\ba$ (Boeing)

**Motivation**

The idea is to encourage you to build re-usable pieces of code.

So if you created some functions in solving Part 1, you may reuse these functions to easily solve part 2,
particulary if you treated the ticker (e.g., $\aapl$ or $\ba$) as a parameter to your functions.

If you simply copy and paste the code from Part 1 you will only get partial credit.


**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [6]:
beta_0 = 0    # The regression parameter for the constant
beta_SPY = 0  # The regression parameter for the return of SPY
ticker = "BA"

print("{t:s}: beta_0={b0:3.2f}, beta_SPY={b1:3.2f}".format(t=ticker, b0=beta_0, b1=beta_SPY))


BA: beta_0=0.00, beta_SPY=0.00


**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [7]:
cross_val_avg = 0

print("{t:s}: Avg cross val score = {sc:3.2f}".format(t=ticker, sc=cross_val_avg) )

BA: Avg cross val score = 0.00


**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [8]:
hedged_num_returns = 0  # Number of returns in hedged series
hedged_first_return = 0 # The return on the earliest date in hedged series
hedged_last_return  = 0 # The return on the latest date in hedged series
hedged_avg_return  = 0  # The average return over the hedged series

ticker="BA"
print("{t:s} hedged returns: There are {num:d} returns. First={first:3.2f}, Last={last:3.2f}, Avg={avg:3.2f}".format(t=ticker,
                                                                                                                    num=hedged_num_returns,
                                                                                                                    first=hedged_first_return, 
                                                                                                                    last=hedged_last_return, 
                                                                                                                    avg=hedged_avg_return))


BA hedged returns: There are 0 returns. First=0.00, Last=0.00, Avg=0.00


# Returns to prices

- You have already computed the predicted returns of $\aapl$ for each date in the test set.
- Create the predicted *price* timeseries for $\aapl$ for the date range in the test set
- Plot (on the same graph) the actual price timeseries of $\aapl$ and the predicted price timeseries.

There is a particular reason that we choose to perform the Linear Regression on returns rather than prices.

It is beyond the scope of this lecture to explain why, but we want to show that we can easily convert
back into prices.

**Question**
Replace the 0 values in the following cell with your answers, and execute the print statements

In [9]:
num_prices = 0  # Number of prices in price series
first_price = 0 # The price on the earliest date in price series
last_price  = 0 # The price on the latest date in price series
avg_price  = 0  # The average price over the price series

ticker="AAPL"
print("{t:s} predicted prices: There are {num:d} prices. First={first:3.2f}, Last={last:3.2f}, Avg={avg:3.2f}".format(t=ticker,
                                                                                                                    num=num_prices,
                                                                                                                    first=first_price, 
                                                                                                                    last=last_price, 
                                                                                                                    avg=avg_price))
 

AAPL predicted prices: There are 0 prices. First=0.00, Last=0.00, Avg=0.00


# Extra credit

The data directory has the prices of many other indices.
- Any ticker in the directory beginning with the letter "X" is an index

Choose *one* index (we'll call it $I$) other than $\spy$ to use as a second feature and compute the Linear Regression

$$
\ret_\aapl^\tp = \beta^T \x + \epsilon_{\aapl}^\tp
$$

where $\x$ is the feature vector
  - $\x^\tp = \begin{pmatrix}
        1 \\
        \ret_\spy^\tp \\
        \ret_I^\tp \\
        \end{pmatrix}$

That is, predict the returns of $\aapl$ in terms of a constant, the returns of $\spy$ and the returns of another index $I$.

**Question**
There is no specified format.  Treat this like an interview question and show off your analytical
and explanatory skills. Be sure to explain how you came about choosing the second index.